In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

# Connecting to OpenAI

The next cell is where we load in the environment variables in your `.env` file and connect to OpenAI.

## Troubleshooting if you have problems:

Head over to the [troubleshooting](troubleshooting.ipynb) notebook in this folder for step by step code to identify the root cause and fix it!

If you make a change, try restarting the "Kernel" (the python process sitting behind this notebook) by Kernel menu >> Restart Kernel and Clear Outputs of All Cells. Then try this notebook again, starting at the top.

Or, contact me! Message me or email ed@edwarddonner.com and we will get this to work.

Any concerns about API costs? See my notes in the README - costs should be minimal, and you can control it at every point. You can also use Ollama as a free alternative, which we discuss during Day 2.

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [3]:
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the Troubleshooting notebook in this folder for full instructions

# Let's make a quick call to a Frontier model to get started, as a preview!

In [4]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! Welcome! I'm glad you're here. How can I assist you today?


## OK onwards with our first project

In [5]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [6]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [7]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [8]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [9]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acqui

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

To give you a preview, the next 2 cells make a rather simple call - we won't stretch the mighty GPT (yet!)

In [10]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [11]:
# To give you a preview -- calling OpenAI with system and user messages:

response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

Oh, a tough one! Let me think... Is it 22? Just kidding! It's 4. 


## And now let's build useful messages for GPT-4o-mini, using a function

In [12]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [13]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a

## Time to bring it together - the API for OpenAI is very simple!

In [14]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [15]:
summarize("https://edwarddonner.com")

"# Summary of Edward Donner's Website\n\nEdward Donner's website serves as a personal portfolio highlighting his interests and professional background. He is a software engineer and co-founder of **Nebula.io**, an AI-focused company dedicated to talent discovery and management. His background includes founding the AI startup **untapt**, which was acquired in 2021.\n\n## Interests\n- Programming and experimenting with LLMs (Large Language Models)\n- DJing and amateur electronic music production\n- Engaging with tech discussions, particularly on platforms like Hacker News\n\n## Recent Announcements\n1. **January 23, 2025**: Resources from an LLM Workshop focused on hands-on experience with Agents.\n2. **December 21, 2024**: Welcome message for the community of SuperDataScientists.\n3. **November 13, 2024**: Resources related to mastering AI and LLM engineering.\n4. **October 16, 2024**: Resources for transitioning from a software engineer to an AI data scientist.\n\nThe website encourage

In [16]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [17]:
display_summary("https://edwarddonner.com")

# Summary of Edward Donner's Website

Edward Donner's website serves as a personal platform where he shares his interests and work related to coding and experimentation with large language models (LLMs). He is the co-founder and CTO of Nebula.io, a company focused on harnessing AI to improve talent discovery and engagement. Ed mentions his previous experience as the founder of an AI startup called untapt, which was acquired in 2021.

## Key Features:
- **Connect Four**: An interactive arena for LLMs that tests their skills in diplomacy and strategy.
- **About Ed**: Insights into his professional background, including his interests in DJing and electronic music, alongside his role in technology and AI.

## News and Announcements:
- **January 23, 2025**: Announced resources for the "LLM Workshop – Hands-on with Agents".
- **December 21, 2024**: Welcomed a new community called "SuperDataScientists".
- **November 13, 2024**: Shared resources for "Mastering AI and LLM Engineering".
- **October 16, 2024**: Provided resources for transitioning from software engineering to AI data science.

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. See the community-contributions folder for a Selenium implementation that gets around this. You'll need to read up on installing Selenium (ask ChatGPT!)

Also Websites protected with CloudFront (and similar) may give 403 errors - many thanks Andy J for pointing this out.

But many websites will work just fine!

In [18]:
display_summary("https://cnn.com")

# CNN Website Summary

The CNN website serves as a comprehensive source for breaking news and updates across various categories, including:

- **US and World News**: Coverage on significant global events, including the Ukraine-Russia war and the Israel-Hamas conflict.
- **Politics**: Insight into US political dynamics and upcoming elections, including developments related to Donald Trump's administration and its impact on international relations.
- **Health**: Reports on health-related issues and studies, with recent findings on artificial sweeteners and their effects.
- **Science and Technology**: Notable achievements in space exploration, including the Curiosity rover’s findings on Mars.
- **Business**: Analysis of financial markets and important business developments, such as tariffs and corporate strategies.
- **Entertainment and Lifestyle**: News on celebrities and cultural events, as well as lifestyle pieces focusing on food, travel, and wellness.

### Recent Highlights
- **Ukraine News**: Ukraine has been launching attacks in newly claimed Russian regions amid setbacks on their home front.
- **Natural Disasters**: A recent earthquake has caused over 1,700 fatalities in Myanmar and Thailand, prompting search and rescue efforts.
  
The site is geared towards delivering timely news and analyses, with features that encourage user engagement and feedback.

## Hands-on Example Using GPT-4o-mini 🖥️
Here’s a simple example where I asked GPT-4o-mini for laptop recommendations:

In [19]:
# Step 1: Create your prompts

system_prompt = "You are a knowledgeable product recommendation assistant. \
You provide tailored product suggestions based on user preferences and needs."

user_prompt = """
I am looking for a product recommendation.
Category: Laptops
Budget: $800 - $1200
Preferred Brand: Dell, HP, or Lenovo
Key Features: Lightweight, long battery life, and fast performance
Please suggest 3 options with a short explanation for each.
"""


# Step 2: Make the messages list

messages = [{"role":"system","content":system_prompt},{"role":"user","content":user_prompt}] # fill this in

# Step 3: Call OpenAI

response = openai.chat.completions.create(
    model = "gpt-4o-mini",
    messages = messages
)

# Step 4: print the result

display(Markdown((response.choices[0].message.content)))

Here are three laptop recommendations that fit your criteria of being lightweight, having long battery life, and offering fast performance, within your budget and preferred brands:

### 1. **Dell XPS 13**
- **Price Range**: Around $1,000 - $1,200
- **Key Features**: 
  - **Lightweight**: Weighs approximately 2.7 pounds, making it easy to carry around.
  - **Long Battery Life**: Offers up to 12-14 hours on a single charge, depending on usage.
  - **Performance**: Equipped with Intel's latest 11th or 12th generation i5/i7 processors, providing excellent speed for multitasking and productivity.

### 2. **HP Envy x360 13**
- **Price Range**: About $850 - $1,100
- **Key Features**: 
  - **Lightweight**: Weighs around 3.0 pounds and features a 2-in-1 design that can be used as both a laptop and a tablet.
  - **Long Battery Life**: Up to 10-15 hours of battery life, making it suitable for long work or study sessions.
  - **Performance**: Comes with Ryzen 5 or 7 processors, which offer fast performance for everyday tasks and creative work.

### 3. **Lenovo Yoga 7i**
- **Price Range**: Approximately $800 - $1,200
- **Key Features**: 
  - **Lightweight**: Roughly 3.0 pounds, easy to handle and travel with.
  - **Long Battery Life**: Can last around 10-12 hours on battery life, depending on usage patterns.
  - **Performance**: Features Intel’s 11th or 12th generation processors, ensuring smooth operation for both work and leisure activities.

Each of these options provides a compelling combination of portability, battery efficiency, and high performance, making them suitable choices for productivity on the go.